In [10]:
import pandas as pd
#import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
from pymongo import MongoClient
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json
import csv

from apscheduler.schedulers.blocking import BlockingScheduler

In [11]:
# Connecting to MongoDB
client = MongoClient('mongo', 27017, username='admin', password='DataMan2019!')

### From csv to json on MongoDB

In [4]:
def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)

In [48]:
#from csv to json
csvFilePath = 'df_sino.csv'
jsonFilePath = 'df_sino.json'
csv_to_json(csvFilePath, jsonFilePath)

In [53]:
with open('df_sino.json') as f:
  data = json.load(f)

In [54]:
#from json to mongodb
client.stockex.sino.insert_many(data)

In [5]:
sino_tweet = list(client.twitter.sino.find({}))
sino_tweet_df = pd.DataFrame(sino_tweet, dtype = float)
sino_tweet_df.head()

,_id,data,location,id,testo,username,replyCount,like,retweet_count,quoteCount,...,lang,url,coordinates,place,followersCount,friendsCount,statusesCount,favouritesCount,listedCount,mediaCount
0,60b2463b10d9bb0d1938e933,2021-05-24 23:56:36+00:00,"Toronto, Canada",1.396978e+18,@themojoguy @benwedge @EmmMacfarlane Especiall...,TheTradeLawGuy,1.0,0.0,0.0,0.0,...,en,https://twitter.com/TheTradeLawGuy/status/1396...,,,358.0,358.0,4803.0,4602.0,6.0,217.0
1,60b2463b10d9bb0d1938e934,2021-05-24 23:53:37+00:00,"Toronto, Canada",1.396978e+18,@themojoguy @benwedge @EmmMacfarlane That can'...,TheTradeLawGuy,1.0,0.0,0.0,0.0,...,en,https://twitter.com/TheTradeLawGuy/status/1396...,,,358.0,358.0,4803.0,4602.0,6.0,217.0
2,60b2463b10d9bb0d1938e935,2021-05-24 23:50:46+00:00,"Goderich, Ontario",1.396977e+18,@sunlorrie BS fear mongering article “As an ex...,summeki,0.0,1.0,1.0,0.0,...,en,https://twitter.com/summeki/status/13969770006...,,,172.0,172.0,5005.0,1163.0,2.0,134.0
3,60b2463b10d9bb0d1938e936,2021-05-24 23:47:57+00:00,Australia,1.396976e+18,@Virg2101 @ROO3sixty @Mooby_Doo @JohnRuddick2 ...,Christo69933840,0.0,0.0,0.0,0.0,...,en,https://twitter.com/Christo69933840/status/139...,,,83.0,83.0,27460.0,8460.0,5.0,8454.0
4,60b2463b10d9bb0d1938e937,2021-05-24 23:43:51+00:00,Australia,1.396975e+18,@ROO3sixty @Virg2101 @Mooby_Doo @JohnRuddick2 ...,Christo69933840,0.0,0.0,0.0,0.0,...,en,https://twitter.com/Christo69933840/status/139...,,,83.0,83.0,27460.0,8460.0,5.0,8454.0


In [6]:
sino_stockex = list(client.stockex.sino.find({}))
sino_stockex_df = pd.DataFrame(sino_stockex, dtype = float)
sino_stockex_df.head()

,_id,Datetime,Open,High,Low,Close,Volume
0,60b2466110d9bb0d19395178,2021-04-08 09:30:00+08:00,19.200001,19.379999,19.040001,19.299999,0.0
1,60b2466110d9bb0d19395179,2021-04-08 09:31:00+08:00,19.219999,19.340000,19.219999,19.340000,56000.0
2,60b2466110d9bb0d1939517a,2021-04-08 09:32:00+08:00,19.320000,19.480000,19.320000,19.440001,70400.0
3,60b2466110d9bb0d1939517b,2021-04-08 09:33:00+08:00,19.420000,19.459999,19.360001,19.420000,43200.0
4,60b2466110d9bb0d1939517c,2021-04-08 09:34:00+08:00,19.480000,19.500000,19.320000,19.459999,74000.0


In [33]:
def daily_integration(tweet_df, stockex_df, hours):
    # Type datetime and timezone
    
    tweet_df['data'] = pd.to_datetime(tweet_df['data'])
    
    stockex_df['Datetime'] = pd.to_datetime(stockex_df['Datetime'])
    stockex_df['Datetime']=stockex_df['Datetime'].dt.tz_localize(None)

    # Time delta for (-8 for timezone +1 for correlation)
    hours_added = datetime.timedelta(hours = hours)
    stockex_df['Datetime']= stockex_df['Datetime'] + hours_added
    
    hours = 24 - hours   #+24
    hours_added = datetime.timedelta(hours = hours)
    tweet_df['data']= tweet_df['data'] + hours_added

    # Type datetime and timezone
    tweet_df['data'] = pd.to_datetime(tweet_df['data'])
    tweet_df['data']=tweet_df['data'].dt.tz_localize(None)

    # Rename
    tweet_df = tweet_df.rename(columns={'data': 'Datetime'})

    # Sort
    tweet_df = tweet_df.sort_values(by="Datetime")

    # Delete hour and minutes
    tweet_df['date'] = pd.to_datetime(tweet_df['Datetime']).dt.date
    
    # Create column for number of day (0 is Sunday, 6 is Saturday)
    tweet_df['day_of_week'] = tweet_df['Datetime'].dt.strftime('%w')
    
    # Add a day on for every Sunday
    hours=+24
    hours_added = datetime.timedelta(hours = hours)

    tweet_df['Datetime'] = np.where(tweet_df['day_of_week'] == '0',tweet_df['Datetime']+hours_added,tweet_df['Datetime'])

    # Add two day on for every Saturday
    hours=+48
    hours_added = datetime.timedelta(hours = hours)
    tweet_df['Datetime'] = np.where(tweet_df['day_of_week'] == '6',tweet_df['Datetime']+hours_added,tweet_df['Datetime'])

    # Sunday on Monday
    tweet_df[tweet_df['day_of_week']=='0']
    
    # Saturday on Monday
    tweet_df[tweet_df['day_of_week']=='6']
    
    #sentiment
    analyzer = SentimentIntensityAnalyzer()
    df =[]
    for sentence in tweet_df['testo']:
        vs = analyzer.polarity_scores(sentence)
        vs_df = pd.DataFrame.from_dict(vs.values(), orient='columns')
        df.append(vs_df[0][3])
    tweet_df['compound'] = df
    tweet_df['compound'].describe()
    
    # Rewrite date for weekend
    tweet_df['date'] = pd.to_datetime(tweet_df['Datetime']).dt.date

    #tweet daily analysis
    company_count=pd.DataFrame(dtype=float)

    # Count tweet by hour
    company_count['N of Tweet']= tweet_df.groupby('date')['id'].count()

    # Count like ecc by hour and divide by number of tweet
    company_count['sum of Nlike']= tweet_df.groupby('date')['like'].sum()
    company_count['median of Nlike']= tweet_df.groupby('date')['like'].median()
    company_count['max of Nlike']= tweet_df.groupby('date')['like'].max()
    company_count['sum of retweet']= tweet_df.groupby('date')['retweet_count'].sum()
    company_count['median of retweet']= tweet_df.groupby('date')['retweet_count'].median()
    company_count['max of retweet']= tweet_df.groupby('date')['retweet_count'].max()
    company_count['sum of reply']= tweet_df.groupby('date')['replyCount'].sum()
    company_count['median of reply']= tweet_df.groupby('date')['replyCount'].median()
    company_count['max of reply']= tweet_df.groupby('date')['replyCount'].max()
    company_count['Comp']= tweet_df.groupby('date')['compound'].sum()/company_count['N of Tweet']
    
    company_count.reset_index(inplace=True)
    
    company_count = company_count.rename(columns={'date': 'Datetime'})
    
    company_count['Datetime'] = pd.to_datetime(company_count['Datetime'])
    company_count['day_of_week'] = company_count['Datetime'].dt.strftime('%w')
    #company_count['Datetime'] = pd.to_datetime(company_count['Datetime']).dt.date
    #company_count['Datetime'] = pd.to_datetime(company_count['Datetime'])
    
    company_count['N of Tweet'] = np.where(company_count['day_of_week'] == '1',company_count['N of Tweet']/3,company_count['N of Tweet'])
    company_count['sum of Nlike'] = np.where(company_count['day_of_week'] == '1',company_count['sum of Nlike']/3,company_count['sum of Nlike'])
    company_count['sum of retweet'] = np.where(company_count['day_of_week'] == '1',company_count['sum of retweet']/3,company_count['sum of retweet'])
    company_count['sum of reply'] = np.where(company_count['day_of_week'] == '1',company_count['sum of reply']/3,company_count['sum of reply'])

    #stock data
    # Delete hour from stock dataset
    stockex_df['Datetime'] = pd.to_datetime(stockex_df['Datetime']).dt.date
    stockex_df = stockex_df.drop_duplicates(subset='Datetime', keep="last")
    stockex_df['Datetime'] = pd.to_datetime(stockex_df['Datetime'])
    
    # Merge stock data and tweet indices 
    df_int = stockex_df.merge(company_count, on='Datetime', how='left', indicator=False)
    df_int.to_csv('int.csv')
    csv_to_json('int.csv', 'int.json')
    with open('int.json') as f:
      data = json.load(f)
    return df_int

In [36]:
def job_daily_function(tweet_df, stockex_df, hours):
    data = daily_integration(tweet_df, stockex_df, hours)
    client.integrated_daily.sino.insert_many(data)

In [21]:
def job_func():
    job_daily_function(sino_tweet_df, sino_stockex_df, -8)

In [25]:
# Scheduler
sched = BlockingScheduler()

try:
    sched.add_job(job_func, 'cron', day_of_week='mon-fri', hour='10-3', minute=30)
    sched.start()

except:
    print("An exception occurred")
    #pass

An exception occurred
